In [1]:
import matplotlib.pyplot as plt
import ML_library        as MLL
import numpy             as np
import pandas            as pd
import torch             as torch
import seaborn           as sns
import json
import os

import sys
sys.path.append('../../UPC')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sns.set_theme()

Although training ranges from 100 to 700 K, predictions are made from 200 to 600 K in order to avoid the misbehaviour of $C_p$ at high temperature.

Ferromagnetic transitions are predicted as any phase transition involving a non-centrosymmmetric space group.

In [2]:
target = 'Fv-parameters'

name = '$F_{v}$ (meV/atom)'

input_folder    = 'models'
pred_DB         = 'Loaded_PT'
path_to_pred_DB = f'../../UPC/MP/{pred_DB}'
target_folder   = f'{input_folder}/{target}'
model_name      = f'{target_folder}/model.pt'

# Whether to plot the harmonic extrapolations of Fv (very time-consuming) or not
plot_extrapolations = False

# Predefined index for temperatures (in graphs)
dropout    = 0.3
temp_index = 4

dpi = 50

# Defining the range of temperatures

Ti = 200
Tf = 500
dT = 50

temperatures = np.arange(Ti, Tf+dT, dT)
temperatures_plot = np.arange(Ti, Tf+dT, 0.1)

# Importing the list of non-centrosymmetric phases

non_centrosymmetric_space_groups = np.loadtxt('input/non-centrosymmetric-phases.txt', dtype='str')

# Compute coefficients

In [3]:
dataset_parameters_name_std = f'{target_folder}/standardized_parameters.json'
predictions_file_name       = f'{input_folder}/predictions_phase_transition.txt'

if os.path.exists(predictions_file_name):
    MP_coefficients = np.loadtxt(predictions_file_name)
else:
    # Load the data from the JSON file
    with open(dataset_parameters_name_std, 'r') as json_file:
        numpy_dict = json.load(json_file)

    # Convert torch tensors to numpy arrays
    dataset_parameters = {}
    for key, value in numpy_dict.items():
        try:
            dataset_parameters[key] = torch.tensor(value)
        except:
            dataset_parameters[key] = value
    
    # Create dataset for predictions
    MP_dataset, MP_labels = MLL.create_predictions_dataset(path_to_pred_DB)
    
    # Standardize properties
    MP_dataset = MLL.standarize_dataset(MP_dataset, dataset_parameters, transformation=dataset_parameters['transformation'])
    
    # Initialize the model
    model = MPL.GCNN(features_channels=MP_dataset[0].num_node_features,
                     pdropout=dropout)
    
    # Moving model to device
    model = model.to(device)
    
    # Load Graph Neural Network model
    model.load_state_dict(torch.load(model_name, map_location=torch.device(device)))
    model.eval()

    # Compute predictions
    MP_coefficients = MLL.make_predictions(MP_dataset, model, dataset_parameters)
    
    # Compute Fv
    #MP_predictions = MLL.compute_Fv(temperatures, MP_coefficients)
    
    # Saving the predictions
    #np.savetxt(predictions_file_name, MP_coefficients)

TypeError: must be real number, not NoneType

# Computing phase transitions

In [ ]:
# Loading dictionary of atomic masses

atomic_masses = {}

with open('input/atomic_masses.dat', 'r') as atomic_masses_file:
    for line in atomic_masses_file:
        (key, mass, _, _, _) = line.split()
        atomic_masses[key] = mass

In [ ]:
stable_file_name   = f'stable_transition_phases.txt'
unstable_file_name = f'unstable_transition_phases.txt'

#if os.path.exists(stable_file_name) and os.path.exists(unstable_file_name):
if False:
    stable_transition_phases   = np.loadtxt(stable_file_name,   dtype=str)
    unstable_transition_phases = np.loadtxt(unstable_file_name, dtype=str)
else:
    # Ordering by families
    indexes      = np.argsort(MP_labels)
    labels       = np.array(MP_labels)[indexes]
    coefficients = MP_coefficients[:, indexes]
    metrics      = np.array(MP_metrics)[indexes]

    # Separating compounds and polymorphes
    compounds = []
    polymorfs = []
    for line in labels:
        # Splitting compound from polymorphes names
        split_line = line.split()

        # Appending that names in their respective lists
        compounds.append(split_line[0])
        polymorfs.append(split_line[1])

    # Loading ground state energies

    gs_energies = []
    for i in range(len(polymorfs)):
        compound = compounds[i]
        polymorf = polymorfs[i]

        EPA_folder = f'{path_to_pred_DB}/{compound}/{polymorf}'
        temp_energy = float(np.loadtxt(f'{EPA_folder}/EPA')) * 1e3  # From eV/atom to meV/atom
        gs_energies.append(temp_energy)

    # Unique compounds (not considering polymorphes)
    unique_compounds, unique_counts = np.unique(compounds, return_counts=True)
    unique_indexes = np.append([0], np.cumsum(unique_counts))  # So it starts with 0

    # Initializing files with data
    stable_transition_phases   = []
    unstable_transition_phases = []
    for i in range(len(unique_compounds)):
        compound = unique_compounds[i]

        # Getting corresponding predictions
        index1 = unique_indexes[i]
        index2 = unique_indexes[i+1]

        static_energies          = gs_energies[index1:index2]
        vibrational_coefficients = coefficients[:, index1:index2]
        vibrational_metrics      = metrics[index1:index2]

        n_polymorfs = index2 - index1
        if n_polymorfs > 1:
            print(compound)
            if plot_extrapolations:
                fig = plt.figure(figsize=(5, 5))
            
            free_energies = np.zeros((n_polymorfs, len(temperatures)))
            for k in range(n_polymorfs):
                _beta_             = vibrational_coefficients[:, k]
                vibrational_energy = MPL.Helmholtz_free_energy_function(temperatures, *_beta_)
                free_energies[k]   = static_energies[k] + vibrational_energy  # F(T) = E0 + F_v(T)
            
            for k in range(n_polymorfs):
                _beta_1          = vibrational_coefficients[:, k]
                fitting_metric_1 = vibrational_metrics[k]
                static_energy_1  = static_energies[k]
                free_energies_1  = free_energies[k]
                polymorf_1       = polymorfs[index1+k]

                # Determine the centrosymmetry of the phase
                non_centrosymmetric_1 = True if polymorf_1 in non_centrosymmetric_space_groups else False
                
                # Plotting
                if plot_extrapolations:
                    # Smooth Gibbs energy for plotting
                    gibbs_energy_plot = static_energy_1 + MPL.Helmholtz_free_energy_function(temperatures_plot,
                                                                                             *_beta_1)
                    plt.plot(temperatures_plot, gibbs_energy_plot, label=polymorf_1)

                # Looking for phase transitions
                for m in np.arange(k+1, n_polymorfs):
                    _beta_2          = vibrational_coefficients[:, m]
                    fitting_metric_2 = vibrational_metrics[m]
                    static_energy_2  = static_energies[m]
                    free_energies_2  = free_energies[m]
                    polymorf_2       = polymorfs[index1+m]
                    
                    # Computing the entropy change provoked by the transition. Parameters were previously scaled
                    d_E0    = static_energy_2 - static_energy_1
                    d_alpha = (_beta_2[0] - _beta_1[0])
                    d_beta  = (_beta_2[1] - _beta_1[1]) * 1e-5
                    d_gamma = (_beta_2[2] - _beta_1[2]) * 1e-10

                    temp_root_1 = (- d_beta + np.sqrt(d_beta**2 - 4 * (d_E0 + d_alpha) * d_gamma)) / (2 * d_gamma)
                    temp_root_2 = (- d_beta - np.sqrt(d_beta**2 - 4 * (d_E0 + d_alpha) * d_gamma)) / (2 * d_gamma)

                    c_temperatures = [np.sqrt(temp_root_1), np.sqrt(temp_root_2)]

                    # Remove those temperature out of the temperature range of interest
                    c_valid_temperatures = [x for x in c_temperatures if (x > Ti and x < Tf)]

                    # Determine if there is any transition within the temperature interval
                    if not len(c_valid_temperatures):  # There is not a transition between these polymorfs
                        continue
                    elif len(c_valid_temperatures) > 1:
                        print(f'Warning: {len(c_valid_temperatures)} transitions within this interval')

                    # Get the expected critical temperature and free energy
                    c_temperature = np.min(c_valid_temperatures)
                    c_free_energy = static_energy_1 + MPL.Helmholtz_free_energy_function(c_temperature, *_beta_1)
                    d_slope       = c_temperature * np.abs(2 * d_beta + 4 * d_gamma * c_temperature**2)
                    
                    # Read atomic information for entropy change estimation
                    with open(f'{path_to_pred_DB}/{compound}/{polymorf_1}/POSCAR', 'r') as POSCAR_file:
                        POSCAR_lines = POSCAR_file.readlines()
                    composition   = POSCAR_lines[5].split()
                    concentration = np.array(POSCAR_lines[6].split(), dtype=float)
                    
                    # Get atomic mass involved in the transition
                    total_mass = 0
                    for idx in range(len(composition)):
                        total_mass += float(atomic_masses[composition[idx]]) * concentration[idx]
                    
                    # From meV/atomK to J/kgK (1e3 as g goes to kg)
                    c_entropy = d_slope * 6.022 * 1.6 * 1e4 * np.sum(concentration) / total_mass
                    
                    # Check which has lower energy at lower temperature
                    c_free_energy_1 = static_energy_1 + MPL.Helmholtz_free_energy_function(c_temperature-5,
                                                                                           *_beta_1)
                    c_free_energy_2 = static_energy_2 + MPL.Helmholtz_free_energy_function(c_temperature-5,
                                                                                           *_beta_2)

                    first_polymorf = polymorf_1
                    last_polymorf  = polymorf_2
                    if c_free_energy_1 > c_free_energy_2:
                        first_polymorf = polymorf_2
                        last_polymorf  = polymorf_1

                    label = f'{first_polymorf} -> {last_polymorf} ({c_temperature:.2f}K, {c_entropy:.2f} J/kgK)'

                    if plot_extrapolations:
                        if len(c_valid_temperatures):
                            plt.plot(c_temperature, c_free_energy, 'o', label=label)
                    
                    # Determine the centrosymmetry of the phase
                    non_centrosymmetric_2 = True if polymorf_2 in non_centrosymmetric_space_groups else False

                    # 0: None are centrosymmetric
                    # 1: One is centrosymmetric
                    # 2: Both are centrosymmetric
                    non_centrosymmetric = non_centrosymmetric_1 + non_centrosymmetric_2

                    # Appending information (compound name, involved phases in order,
                    # temperature of transition, entropy change involved in the transition,
                    # whether a non-centrosymmetric phase is involved or not)
                    
                    # Compound and involved polymorfs, temperature of the transition, entropy change involved,
                    # centrosymmetry and sloped change involved (presumably, higher slope higher probability of
                    # correct prediction)
                    transition_data = [compound,
                                       first_polymorf, last_polymorf,
                                       c_temperature,
                                       c_entropy,
                                       non_centrosymmetric,
                                       d_slope,
                                       fitting_metric_1,
                                       fitting_metric_2]
                    print(transition_data)

                    min_vib_energy = np.inf
                    for j in range(n_polymorfs):
                        static_energy = static_energies[j]
                        vibrational_energy = MPL.Helmholtz_free_energy_function(c_temperature, *vibrational_coefficients[:, j])
                        vib_energy = static_energy + vibrational_energy
                        if vib_energy < min_vib_energy:
                            min_vib_energy = vib_energy
                    
                    if np.isclose(c_free_energy, min_vib_energy):
                        stable_transition_phases.append(transition_data)
                    else:
                        unstable_transition_phases.append(transition_data)

            if plot_extrapolations:
                #if len(c_valid_temperatures):
                plt.title(compound)
                plt.xlabel('$T$ (K)')
                plt.ylabel(f'$F$ (meV/atom)')
                plt.legend(loc=(1.05, 0.5))
                plt.savefig(f'out/{compound}.eps', dpi=50, bbox_inches='tight')
                plt.show()

    #np.savetxt(stable_file_name,   stable_transition_phases,   fmt='%s')
    #np.savetxt(unstable_file_name, unstable_transition_phases, fmt='%s')

# Visualizing the results

In [ ]:
stable_transition_phases = np.array(stable_transition_phases)

stable_critical_temperatures = np.array(stable_transition_phases[:,  3], dtype=float)
stable_entropy_changes       = np.array(stable_transition_phases[:,  4], dtype=float)
stable_non_centrosymmetry    = np.array(stable_transition_phases[:,  5], dtype=float)
stable_slope_changes         = np.array(stable_transition_phases[:,  6], dtype=float)
#stable_fitting_metrics       = np.array(stable_transition_phases[:,  7], dtype=float)

unstable_transition_phases = np.array(unstable_transition_phases)

unstable_critical_temperatures = np.array(unstable_transition_phases[:,  3], dtype=float)
unstable_entropy_changes       = np.array(unstable_transition_phases[:,  4], dtype=float)
unstable_non_centrosymmetry    = np.array(unstable_transition_phases[:,  5], dtype=float)
unstable_slope_changes         = np.array(unstable_transition_phases[:,  6], dtype=float)
#unstable_fitting_metrics       = np.array(unstable_transition_phases[:,  7], dtype=float)

## First N stable materials highest entropy changes

In [ ]:
# [::-1] sorts it from higher to lower
entropy_sorted_stp = stable_transition_phases[stable_entropy_changes.argsort()[::-1]]

# Remove excessively low slope changes
#entropy_sorted_stp = entropy_sorted_stp[np.array(entropy_sorted_stp[:,  6], dtype=float) > 0.2]

# Convert into dataframe and save to excel
columns   = ['Material', 'Phase 1', 'Phase 2', 'T (K)', 'S (J/kgK)', 'Centrosymmetry', 'Slope', 'Fitting-metric-1', 'Fitting-metric-2']
df_es_stp = pd.DataFrame(entropy_sorted_stp, columns=columns)
df_es_stp.to_excel('High-entropy stable transitions.xlsx')
df_es_stp

## Amount of phase transitions at each temperature interval

In [ ]:
plt.hist(unstable_critical_temperatures, alpha=0.4, label='Metastable')
plt.hist(stable_critical_temperatures,   alpha=0.4, label='Stable')
plt.legend(loc='best')
plt.xlabel('T (K)')
plt.ylabel(r'$N_{PT}$')
plt.savefig(f'{input_folder}/Histogram_critical_temperatures.eps', dpi=dpi, bbox_inches='tight')
plt.show()

## Amount of phase transitions involving some entropy change

In [ ]:
plt.hist(unstable_entropy_changes, bins=100, alpha=0.4, label='Metastable')
plt.hist(stable_entropy_changes,   bins=100, alpha=0.4, label='Stable')

plt.legend(loc='best')
plt.xlim(0, 1000)
plt.xlabel(r'$\Delta S$ (J / kg K)')
plt.ylabel(r'$N_{PT}$')
plt.savefig(f'{input_folder}/Histogram_entropy_changes.eps', dpi=dpi, bbox_inches='tight')
plt.show()

## Amount of phase transitions in terms of centrosymmetry

In [ ]:
# Swith to type of transition
plt_unstable_non_centrosymmetry = np.array(unstable_non_centrosymmetry, dtype=object)
plt_stable_non_centrosymmetry   = np.array(stable_non_centrosymmetry,   dtype=object)

centrosymmetries = np.array(['Para-para', 'Ferro-para', 'Ferro-Ferro'])

for i in range(len(centrosymmetries)):
    plt_unstable_non_centrosymmetry[np.where(unstable_non_centrosymmetry == i)[0]] = centrosymmetries[i]
    plt_stable_non_centrosymmetry[np.where(stable_non_centrosymmetry     == i)[0]] = centrosymmetries[i]

# Create the histogram
plt.hist(plt_unstable_non_centrosymmetry, alpha=0.4, label='Metastable', align='left')
plt.hist(plt_stable_non_centrosymmetry,   alpha=0.4, label='Stable', align='left')
plt.ylabel(r'$N_{PT}$')
plt.legend(loc='best')
plt.savefig(f'{input_folder}/Histogram_centrosymmetry.eps', dpi=dpi, bbox_inches='tight')
plt.show()

## Amount of phase transitions at each temperature and entropy change

In [ ]:
x_edges = temperatures
y_edges = np.linspace(0, 500, 10)

distribution = np.histogram2d(stable_critical_temperatures, stable_entropy_changes,
                              bins=(list(x_edges), list(y_edges))
                             )[0].T

X, Y = np.meshgrid(x_edges, y_edges)
plt.pcolormesh(X, Y, distribution, shading='auto', cmap='viridis')
plt.xlabel('T (K)')
plt.ylabel(r'$\Delta S$ (J / kg K)')
plt.savefig(f'{input_folder}/Temperature_vs_stable_entropy_changes.eps', dpi=dpi, bbox_inches='tight')
plt.show()

In [ ]:
x_edges = temperatures
y_edges = np.linspace(0, 500, 10)

distribution = np.histogram2d(unstable_critical_temperatures, unstable_entropy_changes,
                              bins=(list(x_edges), list(y_edges))
                             )[0].T

X, Y = np.meshgrid(x_edges, y_edges)
plt.pcolormesh(X, Y, distribution, shading='auto', cmap='viridis')
plt.xlabel('T (K)')
plt.ylabel(r'$\Delta S$ (J / kg K)')
plt.savefig(f'{input_folder}/Temperature_vs_unstable_stable_entropy_changes.eps', dpi=dpi, bbox_inches='tight')
plt.show()

## Amount of phase transitions at each temperature and centrosymmetries

In [ ]:
# Create a mapping of strings to unique integers
centrosymmetries_dict = {'Para-para':   0,
                         'Ferro-para':  1,
                         'Ferro-Ferro': 2}
centrosymmetry_edges = [0, 0.66, 1.33, 2]
centrosymmetry_ticks_x = [0.33, 1, 1.66]
centrosymmetry_ticks_labels = ['Para-para', 'Ferro-para', 'Ferro-Ferro']

In [ ]:
x_edges = temperatures
y_edges = centrosymmetry_edges

# Convert strings to corresponding integers
plt_stable_non_centrosymmetry_as_int = np.array([centrosymmetries_dict[string] for string in plt_stable_non_centrosymmetry])

distribution = np.histogram2d(stable_critical_temperatures, plt_stable_non_centrosymmetry_as_int,
                              bins=(list(x_edges), list(y_edges))
                             )[0].T

X, Y = np.meshgrid(x_edges, y_edges)
plt.pcolormesh(X, Y, distribution, shading='auto', cmap='viridis')
plt.yticks(centrosymmetry_ticks_x,
           centrosymmetry_ticks_labels)
plt.xlabel('T (K)')
plt.savefig(f'{input_folder}/Temperature_vs_stable_centrosymmetry.eps', dpi=dpi, bbox_inches='tight')
plt.show()

In [ ]:
x_edges = temperatures
y_edges = centrosymmetry_edges

# Convert strings to corresponding integers
plt_unstable_non_centrosymmetry_as_int = np.array([centrosymmetries_dict[string] for string in plt_unstable_non_centrosymmetry])

distribution = np.histogram2d(unstable_critical_temperatures, plt_unstable_non_centrosymmetry_as_int,
                              bins=(list(x_edges), list(y_edges))
                             )[0].T

X, Y = np.meshgrid(x_edges, y_edges)
plt.pcolormesh(X, Y, distribution, shading='auto', cmap='viridis')
plt.yticks(centrosymmetry_ticks_x,
           centrosymmetry_ticks_labels)
plt.xlabel('T (K)')
plt.savefig(f'{input_folder}/Temperature_vs_unstable_centrosymmetry.eps', dpi=dpi, bbox_inches='tight')
plt.show()

## Amount of phase transitions with centrosymmetries and entropy change

In [ ]:
x_edges = centrosymmetry_edges
y_edges = np.linspace(0, 500, 10)

distribution = np.histogram2d(plt_stable_non_centrosymmetry_as_int, stable_entropy_changes,
                              bins=(list(x_edges), list(y_edges))
                             )[0].T

X, Y = np.meshgrid(x_edges, y_edges)
plt.pcolormesh(X, Y, distribution, shading='auto', cmap='viridis')
plt.xticks(centrosymmetry_ticks_x,
           centrosymmetry_ticks_labels)
plt.ylabel(r'$\Delta S$ (J / kg K)')
plt.savefig(f'{input_folder}/Entropy_changes_vs_stable_centrosymmetry.eps', dpi=dpi, bbox_inches='tight')
plt.show()

In [ ]:
x_edges = centrosymmetry_edges
y_edges = np.linspace(0, 500, 10)

distribution = np.histogram2d(plt_unstable_non_centrosymmetry_as_int, unstable_entropy_changes,
                              bins=(list(x_edges), list(y_edges))
                             )[0].T

X, Y = np.meshgrid(x_edges, y_edges)
plt.pcolormesh(X, Y, distribution, shading='auto', cmap='viridis')
plt.xticks(centrosymmetry_ticks_x,
           centrosymmetry_ticks_labels)
plt.ylabel(r'$\Delta S$ (J / kg K)')
plt.savefig(f'{input_folder}/Entropy_changes_vs_unstable_centrosymmetry.eps', dpi=dpi, bbox_inches='tight')
plt.show()